In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import numpy as np
import open3d as o3d
from rlbench.action_modes.action_mode import MoveArmThenGripper
from rlbench.action_modes.arm_action_modes import JointVelocity
from rlbench.action_modes.gripper_action_modes import Discrete
from rlbench.environment import Environment
from rlbench.tasks import ReachTarget

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
action_mode = MoveArmThenGripper(
  arm_action_mode=JointVelocity(),
  gripper_action_mode=Discrete()
)
env = Environment(action_mode, headless=False)
env.launch()

task = env.get_task(ReachTarget)
descriptions, obs = task.reset()
obs, reward, terminate = task.step(np.random.normal(size=env.action_shape))

In [2]:
import sys
sys.path.append('../')

from src.rlbench.enviroment import RLBenchEnv

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
import numpy as np
action_bounds = np.array([-0.3, -0.5, 0.6]), np.array([0.7, 0.5, 1.6])
nbins = 10

In [4]:
env = RLBenchEnv('ReachTarget', action_bounds, nbins)

In [ ]:
trajs = env.get_demos(20)

Keyframes num:  2
Keyframes num:  3
Keyframes num:  3
Keyframes num:  3
Keyframes num:  4
Keyframes num:  3
Keyframes num:  3
Keyframes num:  3
Keyframes num:  2
Keyframes num:  2
Keyframes num:  2
Keyframes num:  2
Keyframes num:  4
Keyframes num:  2
Keyframes num:  2


In [2]:
import jax
import jax.numpy as jnp
jax.config.update('jax_platform_name', 'cpu')
from flax import linen as nn
import tensorflow_probability.substrates.jax  as tfp
tfd = tfp.distributions

In [3]:
logist = jnp.zeros((8, 100))